In [1]:
import torchvision
from PIL import Image
import argparse
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision.transforms import ToPILImage
import time
import sys
import torch.multiprocessing as mp
import os

In [2]:
sys.path.insert(1, 'C:/Users/carlo/Documents/GitHub/Mestrado/Codigo')

In [3]:
mp.cpu_count()
torch.cuda.is_available()
torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650'

In [4]:
from inference import image_haze_removel
import metric  as  metric
import evolution as  evolution

In [5]:
import importlib
import evolution
importlib.reload(evolution)
importlib.reload(torch.multiprocessing)
if __name__ == '__main__':
		mp.set_start_method('spawn',force=True)

In [8]:
import tensorflow as tf
from torchmetrics import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio, MultiScaleStructuralSimilarityIndexMeasure
from inference import image_haze_removal
# from torcheval.metrics import PeakSignalNoiseRatio
import lightdehazeNet

def dehaze_test(image_path,weight):
	metrics = np.asarray([0,0])
	inference_time = 0
	metric_time = 0
	image_conversion_time = 0
	image_load_time = 0
	for input in image_path:
		start = time.time()
		hazy_input_image = Image.open(input)
		end = time.time()
		image_load_time += end-start
		start = time.time()
		dehaze_image = image_haze_removel(hazy_input_image,weight)
		end = time.time()
		inference_time += end-start
		# torchvision.utils.save_image(dehaze_image, "dehaze.jpg")
		start = time.time()
		tensor_image = dehaze_image  # Your tensor image
		tensor_image = tensor_image.cpu()  # Move tensor to CPU if it's on GPU
		tensor_image = tensor_image.squeeze(0)  # Remove the batch dimension if present

		# Convert the tensor to a PIL image
		to_pil = ToPILImage()
		pil_image = to_pil(tensor_image)
		# hazy_input_image.show()
		# pil_image.show()

		hazy_input_image = np.array(hazy_input_image)
		enhanced_image = np.array(pil_image)
		end = time.time()
		image_conversion_time += end-start
		start = time.time()
		metrics = metrics + np.asarray([metric.PSNR(hazy_input_image,enhanced_image),metric.SSIM(hazy_input_image,enhanced_image)])
		end = time.time()
		metric_time += end-start
	print("Inference time:",inference_time)
	print("Metric time:",metric_time)
	print("Image conversion time",image_conversion_time)
	print("Image load time",image_load_time)
	return metrics/len(image_path)

def dehaze_test_preload(image_path,weight):
	metrics = np.asarray([0,0])
	ld_net = lightdehazeNet.LightDehaze_Net().cuda()
	ld_net.load_state_dict(weight)
	for input in image_path:
		hazy_input_image = Image.open(input)
		dehaze_image = image_haze_removal(hazy_input_image,ld_net)
		# torchvision.utils.save_image(dehaze_image, "dehaze.jpg")
		tensor_image = dehaze_image  # Your tensor image
		tensor_image = tensor_image.cpu()  # Move tensor to CPU if it's on GPU
		tensor_image = tensor_image.squeeze(0)  # Remove the batch dimension if present

		# Convert the tensor to a PIL image
		to_pil = ToPILImage()
		pil_image = to_pil(tensor_image)
		# hazy_input_image.show()
		# pil_image.show()

		hazy_input_image = np.array(hazy_input_image)
		enhanced_image = np.array(pil_image)
		metrics = metrics + np.asarray([metric.PSNR(hazy_input_image,enhanced_image),metric.SSIM(hazy_input_image,enhanced_image)])
	return metrics/len(image_path)



def dehaze_test_tf(image_path,weight):
	metrics = np.asarray([0,0])
	for input in image_path:
		hazy_input_image = Image.open(input)
		hazy_input_image_tensor = torchvision.transforms.functional.pil_to_tensor(hazy_input_image)
		#hazy_input_image_tensor = torch.as_tensor(hazy_input_image_tensor, device=torch.device('cuda'))
		dehaze_image = image_haze_removel(hazy_input_image,weight)
		# torchvision.utils.save_image(dehaze_image, "dehaze.jpg")
		#tf_image = torch.as_tensor(dehaze_image, device=torch.device('cuda'))
		hazy_input_image_tensor = tf.transpose(tf.convert_to_tensor(hazy_input_image_tensor.numpy()))
		tf_image = tf.transpose(tf.convert_to_tensor(tf.cast(dehaze_image.cpu().squeeze(0).detach().numpy()*255, tf.uint8)))
		metrics = metrics + np.asarray([tf.image.psnr(hazy_input_image_tensor,tf_image,255),tf.image.ssim(hazy_input_image_tensor,tf_image,255)])
	return metrics/len(image_path)

def dehaze_test_torch(image_path,weight):
	psnr = PeakSignalNoiseRatio().to('cuda:0')
	ssim = MultiScaleStructuralSimilarityIndexMeasure(data_range=1.0).to('cuda:0')
	metrics = np.asarray([0,0])
	for input in image_path:
		hazy_input_image = Image.open(input)
		hazy_input_image_tensor = ((torchvision.transforms.functional.pil_to_tensor(hazy_input_image)).cuda()).unsqueeze(0)
		dehaze_image = image_haze_removel(hazy_input_image,weight)
		# torchvision.utils.save_image(dehaze_image, "dehaze.jpg")
		tf_image = dehaze_image
		tensor_image = dehaze_image  # Your tensor image
		tensor_image = tensor_image.cpu()  # Move tensor to CPU if it's on GPU
		tensor_image = tensor_image.squeeze(0)  # Remove the batch dimension if present

		# Convert the tensor to a PIL image
		to_pil = ToPILImage()
		pil_image = to_pil(tensor_image)
		# hazy_input_image.show()
		# pil_image.show()
		metrics = metrics + np.asarray([psnr(tf_image, hazy_input_image_tensor).cpu().detach().numpy(),ssim(tf_image, hazy_input_image_tensor).cpu().detach().numpy()])
	return metrics/len(image_path)

In [7]:
def list_split(original_list,number_of_splits):
    sub_list = []
    split = []
    number_of_elements = len(original_list)
    split_size = int(number_of_elements/number_of_splits)
    extra_elements = number_of_elements%number_of_splits
    last_element = 0
    for i in range(0,number_of_splits):
        if i < extra_elements:
            sub_list.append(original_list[last_element:last_element+split_size+1])
            last_element = last_element+split_size+1
        else:
            sub_list.append(original_list[last_element:last_element+split_size])
            last_element = last_element+split_size
    return sub_list

In [ ]:
path = "C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/img_teste/"

weight = torch.load('C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/trained_weights/trained_LDNet.pth')

path_list = []
for file in os.listdir(path):
	path_list.append(path+file)

path_split = list_split(path_list,4)

argument_splited=[]
for i in range(0,len(path_split)):
    argument_splited.append((path_split[i],weight))

pool = mp.Pool(processes=4)
start = time.time()
results = pool.map(dehaze_test, argument_splited)
pool.close()
pool.join()
print(results)

In [8]:
path = "C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/img_teste/"

img = path+ "047.jpg"

weight = torch.load('C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/trained_weights/trained_LDNet.pth')
# weight_cpu = torch.load('C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/trained_weights/trained_LDNet.pth',map_location='cpu')

device1 = [	path+ "017.jpg",path+ "018.jpg",path+ "019.jpg",path+ "020.jpg",path+ "021.jpg",path+ "022.jpg",path+ "023.jpg",path+ "024.jpg"]
device2 = [	path+ "001.jpg", path+ "002.jpg",path+ "003.jpg",path+ "004.jpg",path+ "005.jpg",path+ "006.jpg",path+ "007.jpg",path+ "008.jpg",
			path+ "009.jpg",path+ "010.jpg",path+ "011.jpg",path+ "012.jpg",path+ "013.jpg",path+ "014.jpg",path+ "015.jpg",path+ "016.jpg"]
device3 = [	path+ "025.jpg", path+ "026.jpg",path+ "027.jpg",path+ "028.jpg",path+ "029.jpg",path+ "030.jpg"]

device = []
for file in os.listdir(path):
	device.append(path+file)

evolutionProcess = evolution.EvolutionaryProcess(weight,16,4,0.01,5,device3)

optimized_weight = evolutionProcess.return_apex()

In [46]:
import time

In [10]:
start = time.time()
evolutionProcess.evolve()
end = time.time()
print(end - start)

-> Round  0
[[[<PIL.Image.Image image mode=RGB size=778x775 at 0x1C18B5182B0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=778x775 at 0x1C18B52CDF0>], [<PIL.Image.Image image mode=RGB size=987x1200 at 0x1C1F4BB1C10>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=987x1200 at 0x1C18B563C70>]], [[<PIL.Image.Image image mode=RGB size=773x1200 at 0x1C18B518820>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=773x1200 at 0x1C18B563CD0>], [<PIL.Image.Image image mode=RGB size=880x1070 at 0x1C18B518AC0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=880x1070 at 0x1C18B563D60>]], [[<PIL.Image.Image image mode=RGB size=873x1125 at 0x1C18B518AF0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=873x1125 at 0x1C18B563D90>]], [[<PIL.Image.Image image mode=RGB size=1024x842 at 0x1C18B518D30>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x842 at 0x1C18B563DF0>]]]
<class 'list'>
Metric calculation time:  3.988997220993042


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [9]:
importlib.reload(metric)

path = "C:/Users/carlo/Documents/GitHub/ProjetoPecem/imagens_nao_rotuladas/all/"

weight = torch.load('C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/trained_weights/trained_LDNet.pth')

path_list = []
for file in os.listdir(path):
	path_list.append(path+file)

# path_list = [path+"Cam 077.3 - 11.10.2022 - 04h.jpg"]

path = "C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/img_teste/"

path_list = []
for file in os.listdir(path):
	path_list.append(path+file)

#path = "C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/SOTS/indoor/gt/"
#for file in os.listdir(path):
#	path_list.append(path+file)
#path = "C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/SOTS/indoor/hazy/"
#for file in os.listdir(path):
#	path_list.append(path+file)
#path = "C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/SOTS/outdoor/gt/"
#for file in os.listdir(path):
#	path_list.append(path+file)
#path = "C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/SOTS/outdoor/hazy/"
#for file in os.listdir(path):
#	path_list.append(path+file)
start = time.time()
print(dehaze_test_preload(path_list,weight))
end = time.time()
print(end - start)
start = time.time()
print(dehaze_test(path_list,weight))
end = time.time()
print(end - start)



[12.51152623  0.55359644]
23.70296597480774
Inference time: 2.169631242752075
Metric time: 10.881061315536499
Image conversion time 6.598401784896851
Image load time 0.034940481185913086
[12.51152623  0.55359644]
19.68700075149536


In [7]:
from enhancement_algorithms import UM,HEF,CLAHE
importlib.reload(metric)
import cv2

def dehaze_test_CLAHE(image_path):
	metrics = np.asarray([0,0])
	for image_input in image_path:
		hazy_input_image = cv2.imread(image_input,cv2.IMREAD_GRAYSCALE)

		dehaze_image = CLAHE(hazy_input_image,2,8)

		metrics = metrics + np.asarray([metric.PSNR(hazy_input_image,dehaze_image),metric.SSIM(hazy_input_image,dehaze_image)])
	return metrics/len(image_path)

path = "C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/img_teste/"
# path = "C:/Users/carlo/Documents/GitHub/ProjetoPecem/imagens_nao_rotuladas/all/"

weight = torch.load('C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/trained_weights/trained_LDNet.pth')

path_list = []
for file in os.listdir(path):
	path_list.append(path+file)

device = [  path+ "017.jpg",path+ "018.jpg",path+ "019.jpg",path+ "020.jpg",path+ "021.jpg",path+ "022.jpg",
            path+ "023.jpg",path+ "024.jpg",path+ "001.jpg", path+ "002.jpg",path+ "003.jpg",path+ "004.jpg",
            path+ "005.jpg",path+ "006.jpg",path+ "007.jpg",path+ "008.jpg",path+ "009.jpg",path+ "010.jpg",
            path+ "011.jpg",path+ "012.jpg",path+ "013.jpg",path+ "014.jpg",path+ "015.jpg",path+ "016.jpg",
            path+ "025.jpg", path+ "026.jpg",path+ "027.jpg",path+ "028.jpg",path+ "029.jpg",path+ "030.jpg"]
    
# path_list = [path+"Cam 077.3 - 11.10.2022 - 04h.jpg"]
path_list = []
path = "C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/SOTS/indoor/gt/"
for file in os.listdir(path):
	path_list.append(path+file)
path = "C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/SOTS/indoor/hazy/"
for file in os.listdir(path):
	path_list.append(path+file)
path = "C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/SOTS/outdoor/gt/"
for file in os.listdir(path):
	path_list.append(path+file)
path = "C:/Users/carlo/Documents/GitHub/Mestrado/Codigo/SOTS/outdoor/hazy/"
for file in os.listdir(path):
	path_list.append(path+file)

print(dehaze_test_CLAHE(path_list))

[20.93990001  0.85560125]


(3, 2)